In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import wandb
warnings.filterwarnings('ignore')

device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

sweep_config = { "name": "Maximize Accuracy", "method": "bayes" }
sweep_config["metric"] = {
    "name": "val_accuracy",
    "goal": "maximize"
}
parameters = {
    "dropout": {"values": [0, 0.2, 0.3]},
    "learning_rate": { "values": [0.1, 0.05] },
    "growth_rate": { "values": [ 12, 18 ] },
    "bottleneck": { "values": [True, False]},
    "optimizer": { "values": ["SGD", "Adam", "RMSprop" ] },
    "lr_gamma": { "values": [ 0.1, 0.3 ] },
    "temperature": { "values": [ 0.3, 0.5 ] },
    "weight_decay": { "values": [4e-5, 1e-5] },
    "alpha": { "values": [ 0.5, 0.7, 0.9 ]}
}
sweep_config["parameters"] = parameters

additional_config = {
    "num_epochs": 5,
    "workers": 4,
    "batch_size": 96,
    "momentum": 0.9,
    "lr_step_size": 20
}

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(0.5),       
    transforms.ToTensor(),    
    transforms.Normalize((0.5070754, 0.48655024, 0.44091907), (0.26733398, 0.25643876, 0.2761503))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5070754, 0.48655024, 0.44091907), (0.26733398, 0.25643876, 0.2761503))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

# Split the dataset into training and validation sets
train_dataset, test_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=2)

Using mps device
Files already downloaded and verified
Files already downloaded and verified


In [2]:
class DistillationLoss(nn.Module):
    def __init__(self, alpha, temperature):
        super(DistillationLoss, self).__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, targets):
        student_loss = self.criterion(student_logits, targets)
        
        distillation_loss = F.kl_div(
            F.log_softmax(student_logits / self.temperature, dim=1),
            F.softmax(teacher_logits / self.temperature, dim=1),
            reduction='batchmean'
        ) * (self.temperature ** 2)

        loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss
        return loss

In [3]:
def distill(dataloader, teacher, model, epoch, config):
    size = len(dataloader.dataset)
    model.train()
    distillation_criterion = DistillationLoss(config["alpha"], config["temperature"])
    if config["optimizer"] != "Adam":
        optimizer = getattr(torch.optim, config["optimizer"])(model.parameters(), lr=config["learning_rate"], momentum=config["momentum"], weight_decay=config["weight_decay"])
    else:
        optimizer = getattr(torch.optim, config["optimizer"])(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config["lr_step_size"], gamma=config["lr_gamma"])

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            teacher_logits = teacher(X)
        student_logits = model(X)
        loss = distillation_criterion(student_logits, teacher_logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f"loss: {loss.item():.6f} [{batch * len(X)}/{size}]")

    scheduler.step()

def train(dataloader, teacher, model, config):
    for epoch in range(config["num_epochs"]):
        distill(dataloader, teacher, model, epoch, config)
        test(val_loader, model, nn.CrossEntropyLoss(), epoch)
        

def test(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    step = epoch * len(dataloader.dataset)
    correct /= size
    wandb.log({"epoch": epoch + 1, "val_loss": test_loss, "val_accuracy": 100*correct})
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [4]:
from student import densenet
from teacher import wide_resnet

def train_test_model(config=None):
    with wandb.init(config = config):
        config = wandb.config
        config.update(additional_config)
        run_name_template = "{} / lr_{} / alpha_{} / dropout_{} / growth_{} / bottleneck_{} / lr_gamma_{} / temp_{} / weight_decay_{}"
        run_name = run_name_template.format(config["optimizer"], config["learning_rate"], config["alpha"],
                                            config["dropout"], config["growth_rate"],
                                            config["bottleneck"], config["lr_gamma"],
                                            config["temperature"], config["weight_decay"])
        print("Run name: ", run_name)
        wandb.run.name = run_name

        teacher_model = wide_resnet().to(device)
        teacher_model.eval()
        
        student_model = densenet().to(device)
        student_model.train()

        train(train_loader, teacher_model, student_model, config)


sweep_id = wandb.sweep(sweep_config, project="distillation")
wandb.agent(sweep_id, train_test_model, count=25)

Create sweep with ID: sk61moht
Sweep URL: https://wandb.ai/ns24z093/distillation/sweeps/sk61moht


wandb: Agent Starting Run: 2crdyhfh with config:
wandb: 	alpha: 0.9
wandb: 	bottleneck: False
wandb: 	dropout: 0.2
wandb: 	growth_rate: 18
wandb: 	learning_rate: 0.1
wandb: 	lr_gamma: 0.3
wandb: 	optimizer: SGD
wandb: 	temperature: 0.5
wandb: 	weight_decay: 1e-05
wandb: Currently logged in as: ns24z093. Use `wandb login --relogin` to force relogin


Run name:  SGD / lr_0.1 / alpha_0.9 / dropout_0.2 / growth_18 / bottleneck_False / lr_gamma_0.3 / temp_0.5 / weight_decay_1e-05
loss: 4.343318 [0/40000]
loss: 3.859285 [6400/40000]
loss: 3.709062 [12800/40000]
loss: 3.670708 [19200/40000]
loss: 3.487759 [25600/40000]
loss: 3.308788 [32000/40000]
loss: 3.315468 [38400/40000]
Test Error: 
 Accuracy: 16.1%, Avg loss: 3.534078

loss: 3.041099 [0/40000]
loss: 3.071847 [6400/40000]
loss: 3.053597 [12800/40000]
loss: 2.859319 [19200/40000]


wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)


loss: 2.759206 [25600/40000]


...

Run name:  SGD / lr_0.1 / alpha_0.9 / dropout_0.3 / growth_18 / bottleneck_True / lr_gamma_0.3 / temp_0.5 / weight_decay_1e-05
loss: 4.218929 [0/40000]
loss: 3.811833 [6400/40000]
loss: 3.771649 [12800/40000]
loss: 3.607079 [19200/40000]
loss: 3.624594 [25600/40000]
loss: 3.457611 [32000/40000]
loss: 3.083219 [38400/40000]
Test Error: 
 Accuracy: 12.1%, Avg loss: 3.733317

loss: 3.021126 [0/40000]
loss: 3.290119 [6400/40000]
loss: 2.910751 [12800/40000]
loss: 2.954216 [19200/40000]
loss: 2.877865 [25600/40000]
loss: 3.066600 [32000/40000]
loss: 2.294977 [38400/40000]
Test Error: 
 Accuracy: 26.2%, Avg loss: 2.944865

loss: 2.651605 [0/40000]
loss: 2.574262 [6400/40000]
loss: 2.223831 [12800/40000]
loss: 2.405437 [19200/40000]
loss: 2.508388 [25600/40000]
loss: 2.716457 [32000/40000]
loss: 1.895089 [38400/40000]
Test Error: 
 Accuracy: 36.0%, Avg loss: 2.440603

loss: 2.072308 [0/40000]
loss: 2.170182 [6400/40000]
loss: 2.356375 [12800/40000]
loss: 2.192872 [19200/40000]
loss: 2.114227 

epoch,▁▃▅▆█
val_accuracy,▁▄▆▇█
val_loss,█▅▃▂▁
epoch,5
val_accuracy,48.7525
val_loss,1.85554


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: b2c9gbbb with config:
wandb: 	alpha: 0.7
wandb: 	bottleneck: False
wandb: 	dropout: 0.2
wandb: 	growth_rate: 18
wandb: 	learning_rate: 0.05
wandb: 	lr_gamma: 0.3
wandb: 	optimizer: SGD
wandb: 	temperature: 0.3
wandb: 	weight_decay: 4e-05


Run name:  SGD / lr_0.05 / alpha_0.7 / dropout_0.2 / growth_18 / bottleneck_False / lr_gamma_0.3 / temp_0.3 / weight_decay_4e-05
loss: 3.354159 [0/40000]
loss: 3.073051 [6400/40000]
loss: 3.243021 [12800/40000]
loss: 2.685098 [19200/40000]
loss: 2.831759 [25600/40000]
loss: 2.454076 [32000/40000]
loss: 2.743147 [38400/40000]
Test Error: 
 Accuracy: 16.7%, Avg loss: 3.434104

loss: 2.464139 [0/40000]
loss: 2.520153 [6400/40000]
loss: 2.392344 [12800/40000]
loss: 2.465183 [19200/40000]
loss: 2.380396 [25600/40000]
loss: 2.419028 [32000/40000]
loss: 2.063451 [38400/40000]
Test Error: 
 Accuracy: 26.6%, Avg loss: 2.883815

loss: 2.087478 [0/40000]
loss: 2.206781 [6400/40000]
loss: 2.372774 [12800/40000]
loss: 1.821325 [19200/40000]
loss: 1.840449 [25600/40000]
loss: 1.933722 [32000/40000]
loss: 1.756778 [38400/40000]
Test Error: 
 Accuracy: 32.2%, Avg loss: 2.600117

loss: 2.031932 [0/40000]
loss: 2.091281 [6400/40000]
loss: 2.012183 [12800/40000]
loss: 1.842877 [19200/40000]
loss: 1.66245

epoch,▁▃▅▆█
val_accuracy,▁▄▅▇█
val_loss,█▅▃▂▁
epoch,5
val_accuracy,40.89
val_loss,2.15947
